<a href="https://colab.research.google.com/github/darja/NeuralUni/blob/master/12_classic_ml_light_ryazhskikh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классическое машинное обучение: алгоритмы и этапы обработки данных

Для базы sklearn breast_cancer
1. Сделать классификацию с помощью SVM с rbf kernel, поварьировать параметр gamma. Можно в логарифмическом маcштабе, например от 0.00001 до 0.5. Использовать признаки как исходные, так и преобразованные после нормировки и метода главных компонент. Поварьировать количество компонент.
2. Сделать классификацию с помощью AdaBoostClassifier из sklearn.ensemble. Поварьировать параметр n_estimators. Можно использовать только исходные признаки. Разбивку на обучающую и тестовую выборки сохранять от модели к модели, как сделано в этом ноутбуке.
В отчете указать Accuracy и ошибки первого и второго рода для всех вариантов. Получился ли результат, лучший, чем в ноубтуке?
3. (Опционально) На базе sklearn diabetes сделать регрессию с помощью LASSO. Поварьировать alpha. В отчет - RMSE и какие какие признаки использовались (ненулевые coef_)

# Подготовка окружения

In [0]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import random
import numpy
import itertools

numpy.random.seed(2)

## Загрузка исходных данных

In [0]:
data = datasets.load_breast_cancer()

target = data.target
# меняем индексы классификации:
# 0 - доброкачественная
# 1 - злокачественная
target = 1 - target

### Демонстрация исходных данных

In [0]:
# print("Features: ", data.feature_names)
# print("Labels: ", data.target_names)
print(data.data.shape)
print(data.data[10:14,:])
print(data.target)

### Разбиение на обучающую и тестовую выборки

In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(data.data, target, test_size=0.3)

### Нормирование

In [7]:
print("Features shape: ", features_train.shape)
mean = features_train.mean(0)
std = features_train.std(0)
print("Mean shape: ", mean.shape)

#нормируем обучающую выборку
features_train_prepared = np.zeros_like(features_train)
for i in range(0, features_train.shape[0]):
    features_train_prepared[i,:]= (features_train[i,:]-mean)/std
    
# нормируем тестовую выборку
features_test_prepared  = np.zeros_like(features_test)
for i in range(0,features_test.shape[0]):
    features_test_prepared[i,:]= (features_test[i,:]-mean)/std


Features shape:  (398, 30)
Mean shape:  (30,)


# Классификация

## Утилиты

In [0]:
#вычисление ошибок первого и второго рода
def calculate_errors(expected, predicted):    
    acc = metrics.accuracy_score(expected, predicted)
    FP = sum((expected == 0) & (predicted == 1))
    FN = sum((expected == 1) & (predicted == 0))
    TP = sum((expected == 1) & (predicted == 1))
    TN = sum((expected == 0) & (predicted == 0))
    FPR = FP / (FP + TN)
    FNR = FN / (TP + FN)
    return (FPR, FNR, acc)

def print_errors(expected, predicted):    
    FPR, FNR, acc = calculate_errors(expected, predicted)
    print("Accuracy:", acc)
    print('ERR1 ', FPR)
    print('ERR2 ', FNR)
    return (FPR, FNR, acc)
    
def fit_and_classify(clf, normalized = False):
    train_features = features_train_prepared if normalized else features_train
    test_features = features_test_prepared if normalized else features_test
    
    clf.fit(train_features, labels_train)
    labels_test_predicted = clf.predict(test_features)
    print_errors(labels_test, labels_test_predicted)
    

## Классификация методом опорных векторов

### Базовая классификация

In [20]:
print("--- Kernel: Linear")
clf = SVC(kernel = 'linear')

print("Raw")
fit_and_classify(clf)

print("\nNormalized")
fit_and_classify(clf, normalized = True)

print("\n\n--- Kernel: RBF")
clf = SVC()

print("Raw")
fit_and_classify(clf)

print("\nNormalized")
fit_and_classify(clf, normalized = True)

--- Kernel: Linear
Raw
Accuracy: 0.9532163742690059
ERR1  0.028846153846153848
ERR2  0.07462686567164178

Normalized
Accuracy: 0.9707602339181286
ERR1  0.038461538461538464
ERR2  0.014925373134328358


--- Kernel: RBF
Raw
Accuracy: 0.6081871345029239
ERR1  0.0
ERR2  1.0

Normalized
Accuracy: 0.9766081871345029
ERR1  0.028846153846153848
ERR2  0.014925373134328358


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


### Подбор параметров RBF kernel

In [26]:
def rbfBruteForce(normalized = False):
    train_features = features_train_prepared if normalized else features_train
    test_features = features_test_prepared if normalized else features_test
    
#     gamma_range = np.linspace(1e-6, 0.5, num = 25)
    gamma_range = np.logspace(-6, 0, num = 50)
    C_range = np.linspace(1e-1, 1e+1, num = 50)
    
    # FPR - False Positive Rate
    # FNR - False Negative Rate
    # TFR - Total False Rate
    result = pd.DataFrame(columns=['C', 'Gamma', 'FPR', 'FNR', 'TFR', "Accuracy"])
    
    for gamma in gamma_range:
        for C in C_range:
            clf = SVC(C = C, gamma = gamma)
            clf.fit(train_features, labels_train)

            labels_test_predicted = clf.predict(test_features)
            FPR, FNR, acc = calculate_errors(labels_test, labels_test_predicted)
            
            result = result.append({'C': C, 'Gamma': gamma, 'FPR': FPR, 'FNR': FNR, 'TFR': FPR + FNR, 'Accuracy': acc}, ignore_index=True)
            
    print("\nBest values sorted by Total + False Positive")
    print(result.sort_values(by=['TFR', 'FPR']).head())

    print("\nBest values sorted by Total + False Negative")
    print(result.sort_values(by=['TFR', 'FNR']).head())
    
print("Raw")
rbfBruteForce(normalized = False)

print("\n\nNormalized")
rbfBruteForce(normalized = True)

Raw

Best values sorted by Total + False Positive
             C     Gamma       FPR       FNR       TFR  Accuracy
801   0.302041  0.000091  0.038462  0.089552  0.128014  0.941520
1050  0.100000  0.000373  0.038462  0.089552  0.128014  0.941520
800   0.100000  0.000091  0.048077  0.089552  0.137629  0.935673
802   0.504082  0.000091  0.048077  0.089552  0.137629  0.935673
803   0.706122  0.000091  0.048077  0.089552  0.137629  0.935673

Best values sorted by Total + False Negative
             C     Gamma       FPR       FNR       TFR  Accuracy
801   0.302041  0.000091  0.038462  0.089552  0.128014  0.941520
1050  0.100000  0.000373  0.038462  0.089552  0.128014  0.941520
800   0.100000  0.000091  0.048077  0.089552  0.137629  0.935673
802   0.504082  0.000091  0.048077  0.089552  0.137629  0.935673
803   0.706122  0.000091  0.048077  0.089552  0.137629  0.935673


Normalized

Best values sorted by Total + False Positive
              C     Gamma  FPR       FNR       TFR  Accuracy
1446

## Классификация с помощью AdaBoostClassifier

### Базовая классификация

In [41]:
bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50)
fit_and_classify(bdt)

Accuracy: 0.9532163742690059
ERR1  0.038461538461538464
ERR2  0.05970149253731343


### Подбор параметров

In [44]:
def adaBoostBruteForce(normalized = False):
    train_features = features_train_prepared if normalized else features_train
    test_features = features_test_prepared if normalized else features_test
    
    depth_range = range(1, 5, 1)
    algorithms = ['SAMME', 'SAMME.R']
    n_estimators = [10, 50, 100, 200, 250, 500]

    combinations = list(itertools.product(depth_range, algorithms, n_estimators))

    # FPR - False Positive Rate
    # FNR - False Negative Rate
    # TFR - Total False Rate
    result = pd.DataFrame(columns=['Depth', 'Algorithm', 'n_estimators', 'FPR', 'FNR', 'TFR', "Accuracy"])
    
    for config in combinations:
        depth = config[0]
        alg = config[1]
        n_est = config[2]
        
        bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth), algorithm=alg, n_estimators=n_est)
        bdt.fit(train_features, labels_train)
        
        labels_test_predicted = bdt.predict(test_features)
        FPR, FNR, acc = calculate_errors(labels_test, labels_test_predicted)
        
        result = result.append({'Depth': depth, 'Algorithm': alg, 'n_estimators': n_est, 'FPR': FPR, 'FNR': FNR, 'TFR': FPR + FNR, 'Accuracy': acc}, ignore_index=True)
            
    print("\nBest values sorted by Total + False Positive")
    print(result.sort_values(by=['TFR', 'FPR']).head())

    print("\nBest values sorted by Total + False Negative")
    print(result.sort_values(by=['TFR', 'FNR']).head())

print("Raw")
adaBoostBruteForce(normalized = False)

print("\n\nNormalized")
adaBoostBruteForce(normalized = True)

Raw

Best values sorted by Total + False Positive
   Depth Algorithm n_estimators       FPR       FNR       TFR  Accuracy
4      1     SAMME          250  0.009615  0.029851  0.039466  0.982456
5      1     SAMME          500  0.009615  0.029851  0.039466  0.982456
3      1     SAMME          200  0.019231  0.029851  0.049082  0.976608
11     1   SAMME.R          500  0.019231  0.029851  0.049082  0.976608
25     3     SAMME           50  0.028846  0.029851  0.058697  0.970760

Best values sorted by Total + False Negative
   Depth Algorithm n_estimators       FPR       FNR       TFR  Accuracy
4      1     SAMME          250  0.009615  0.029851  0.039466  0.982456
5      1     SAMME          500  0.009615  0.029851  0.039466  0.982456
3      1     SAMME          200  0.019231  0.029851  0.049082  0.976608
11     1   SAMME.R          500  0.019231  0.029851  0.049082  0.976608
25     3     SAMME           50  0.028846  0.029851  0.058697  0.970760


Normalized

Best values sorted by Tota